# Final Project

## Part 1

The cell below is for imports. You are only allowed to use `numpy` for this part.No additional imports are allowed

In [ ]:
import numpy as np
from mpi4py import MPI

comm   = MPI.COMM_WORLD
nprocs = comm.Get_size()
rank   = comm.Get_rank()

Now, we will create our class `SparseMatrix`

The class will represent a sparse matrix in `COO` format.\
it should also keep track of the shape of the matrix.\
You need to add the necessary attributes to your class to account for the aforementioned requirements

Let's start with the `__init__` method of our class:\
it should take one additional argument `arg` that will represent the various objects from which we can instantiate our class.

First, we should be able to construct an instance of our class from a regular `numpy` 2d array. \
Inside the `__init__` method, check if `arg` is an instance of a `numpy` array. \
Then, check if the provided array represents a valid matrix.\
If it is not the case, an exception should be raised 

In [ ]:
class SparseMatrix:    
    def __init__(self, arg):

Next, we should be able to construct an instance of our class from a tuple of 3 `numpy` arrays representing a matrix in `COO` format (x, Y, Values)\
Extend the __init__ method by checking if arg is an instance of this case.
Then, check if the provided array represents a valid matrix.
If it is not the case, an exception should be raised

In [ ]:
class SparseMatrix:    
    def __init__(self, arg):

Create a function `cooTranspose` that takes an instance of our class `SparseMatrix` and returns its transpose.

In [ ]:
def cooTranspose(a):

Create a function `cooMatVec` that takes an instance of our class `SparseMatrix` and a vector \ 
as a `numpy` array and returns their product.

In [ ]:
def cooMatVec(A, x):

create a function cooMatMat that takes two instances of our class `SparseMatrix` and \ 
returns their matrix product as a 2 dimentional numpy array

In [ ]:
def cooMatMat(A, B):

# Part 2

In this part, we will be solving a system of linear equations involving a sparse matrix `A` in parallel. \
You will not have to solve the system. However, you will have to implement the function `CreateLocalMatVec` that sets the system for the class `LinearSystem`. \
At the end, compare the results and explain any discrepancies.

In [ ]:
#Create the matrix and Rhs
np.random.seed(42)
from scipy.sparse import random
if rank == 0:
    n = 100
    # Set parameters for the sparse matrix
    density = 0.3 # density of non-zero elements (between 0 and 1)
    A =  random(n, n, density=density, format='csr')
    x = np.random.rand(n)
    B_all = np.dot(A.toarray(),x)
else:
    A = None
    B_all = None

In [ ]:
def CreateLocalMatVec(A, B_all):
    if rank == 0:
        
        
        shape = A.shape
        nrows = shape[0]
        # split the number of rows evenly (as possible) among the MPI tasks
        N_pertask, extra =
    
        # count: the size of each sub-task
        count =
    
        # displacement: the starting index of each sub-task
        displ =
    
        #---- Send the relevant subsets of A and B to each slave MPI task ----
        for i in range(1,nprocs):
    
            # Get the start and end row index for this MPI task
            rstart = 
            rend   = 
    
            #---- Get the subsets of A and B using these rows ----
            A_indptr  =  # modified row-pointer array that will be consistent on the MPI task
            
            pstart    =     # starting row-pointer index
            pend      =       # end      row-pointer index
    
            A_indices = 
            A_data    = 
            B         = 
    
            # Save the lengths of each array
            lengths = {
                    'A_indptr' : 
                    'A_indices': 
                    'A_data'   : 
                    'B'        : 
                    }
    
            # Send the arrays and their lenghts to the relevant MPI task
            comm.send()
            comm.Send()
            comm.Send()
            comm.Send()
            comm.Send()
    
        #---- Set the relevant subsets of A and B for the master MPI task (we don't need to do an MPI Send)
        rstart = 
        rend   = 
    
        A_indptr  = 
        pstart    = 
        pend      = 
        A_indices = 
        A_data    = 
        B         = 
    
    else:
        # Receive the array lengths
        lengths   = 
        # Initialise the buffers
        A_indptr  = np.empty()
        A_indices = np.empty()
        A_data    = np.empty()
        B         = np.empty()
        # Receive the arrays
        comm.Recv()
        comm.Recv()
        comm.Recv()
        comm.Recv()
    
        shape = 
    
    #broadcast shape
    shape = 

        
    return (A_indptr, A_indices, A_data, B, shape)

In [ ]:
class LinearSystem():
    def __init__(self, A=None, shape=None, rhs=None, solver=None, comm=None):
        
        from petsc4py import PETSc
        self.opts = PETSc.Options()

        self.ksp = PETSc.KSP()
        self.ksp.create()
        
        mat = PETSc.Mat().createAIJ(comm=comm, size=shape, csr=A)
        mat.setUp()
        mat.assemblyBegin()
        mat.assemblyEnd()
        self.sol, self.rhs = mat.getVecs()
        self.rhs.setArray(rhs)
        
        
        #---- Set up solver -----
        self.ksp = PETSc.KSP().create(comm=comm)
        
        # It is commonly used with the direct solver preconditioners like PCLU and PCCHOLESKY
        self.ksp.setType('preonly')
        pc = self.ksp.getPC()
        pc.setType('lu')
        pc.setFactorSolverType(solver)
        
        self.ksp.setOperators(mat)
        self.ksp.setFromOptions() # Apply any command line options
        self.ksp.setUp()
    
    def solve(self): 
        # st=timeit.default_timer()
        self.ksp.solve(self.rhs, self.sol)

In [ ]:
(A_indptr, A_indices, A_data, b, shape) = CreateLocalMatVec(A, B_all)

L = LinearSystem(A=(A_indptr, A_indices, A_data), rhs=b, shape=shape, solver="mumps")
ts = MPI.Wtime()
L.solve()
te = MPI.Wtime()
tt = comm.reduce(te - ts, op=MPI.MAX, root=0)
if rank == 0:
    print("Timing to solve the linear system with petsc", tt)

In [ ]:
#---- Gather the solution onto a single array on the master MPI task
if rank == 0:
    X = np.empty(shape[0],dtype=np.double)
else:
    X = None
comm.Gatherv(L.sol.array,X)

In [ ]:
np.random.seed(42)

if rank == 0:
    B = SparseMatrix(A.toarray())
    
    from scipy import sparse
    from scipy.sparse.linalg import spsolve
    mat = sparse.csr_matrix((B.vals, (B.rows, B.cols)))
    ts = MPI.Wtime()
    sol = spsolve(mat, b)
    te = MPI.Wtime()
    print("Timing to solve the linear system with spsolve", te - ts)

In [ ]:
if rank == 0:
    print(np.allclose(X,sol))

Put your response below: